In [21]:
import re
import os
import numpy as np
import json

In [23]:
##This script outputs a JSON dictionary where keys = pdb id and values = nested list of CC regions defined by 
##chain ID first residue and last residue of CC region.
def get_CCregion(line_withCCregion):
    #Takes line and extracts and returns the coiled coil chain, starting residue and last residues
    count = 0
    first_residue = []
    last_residue = []
    chain = []
    isitfirst_residue = False
    chainnameover = False
    for x in reversed(line_withCCregion):
        if x == ' ':
            break
        elif x == '-':
            isitfirst_residue = True
        elif x == ':':
            chainnameover = True
        elif x.isdigit() and isitfirst_residue == True:
            first_residue.append(x)
        elif x.isdigit():
            last_residue.append(x)
        elif count != 0:
            chain.append(x)
        else:
            count += 1
    first_residue.reverse()
    last_residue.reverse()
    chain.reverse()
    first_residue_str = ''.join(map(str,first_residue))
    last_residue_str = ''.join(map(str,last_residue))
    chain_str = ''.join(map(str,chain))
    
    return chain_str, first_residue_str, last_residue_str

In [24]:
import_directory = os.fsencode('./')
cc_regions = {}
for socket_file in sorted(os.listdir(import_directory)):
    filename = os.fsdecode(socket_file)
    directory = os.fsdecode(import_directory)
    if os.path.isfile(directory+filename) and not filename.startswith('.') :
        with open(directory+filename, 'r') as socket_output:
                ##numbering will differ from input as blank lines are stripped
                socket_lines = [line for line in socket_output.readlines() if line.strip()]
                ##extract lines that assigned CC regions
                if socket_file[2:6] not in cc_regions:
                    
                    
                    cc_regions[str(socket_file[-8:-4].decode("utf-8"))] = [(get_CCregion(x)) for x in socket_lines if x[0:6] == 'assign']
                    socket_output.close
                else:
                    cc_regions[str(socket_file[-8:-4].decode("utf-8"))].append([(get_CCregion(x)) for x in socket_lines if x[0:6] == 'assign'])
                    socket_output.close
    else:
        pass

In [25]:
with open("./CCregions2020.json", "w") as outfile:  
    json.dump(cc_regions, outfile) 